In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import re
import time
import math

In [ ]:
PAUSE_TIME = 2
driver = webdriver.Chrome(r'C:\Users\Chun\chromedriver_win32\chromedriver.exe')

# create initial urls for each clothing categories
category_list = ['activewear','dresses','jeans','lingerie-sleepwear','tops','workwear']
for category in category_list:
    link = f'https://www.saksfifthavenue.com/c/women-s-apparel/{category}?prefn1=isSale&prefv1=Sale'
    driver.get(link)
    time.sleep(PAUSE_TIME)

#     add click popup     
#     driver.find_element_by_id('welcome-email-modal').click()
#     

    # get the total pages for the current category
    num_item = driver.find_element_by_xpath('//span[@class="search-count d-none d-lg-inline-block"]').text
    num_item = int(re.search('(\d+)',num_item.replace(',','')).group(1))
    num_page = math.ceil(num_item/96)
    
    # redirect to each page under the current category
    for i in range(num_page):
        page_link = f'{link}&start={96*i}&sz=24'
        driver.get(page_link)
        time.sleep(PAUSE_TIME)    
    
        #scroll down until the end of current page
        while True:
            last_height = driver.execute_script("return document.body.scrollHeight")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")
        
            if new_height == last_height:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(PAUSE_TIME)
                new_height = driver.execute_script("return document.body.scrollHeight")

                if new_height == last_height:
                    break
                else:
                    last_height = new_height
                    continue
        
        #get the product links on the current page  
        item_links = driver.find_elements_by_class_name("link")

        product_links = []
        for i in range(len(item_links)):
            if i%2 == 0:
                item_link = links[i].get_attribute("href")
                product_links.append(item_link)
        
        #enter each product page for product information
        for link in product_links:
        driver.get(link)
        time.sleep(PAUSE_TIME)
        
        ## Make csv_writer ##
        csv_file = open('off5th.csv', 'w', encoding='utf-8', newline='')
        writer = csv.writer(csv_file)

        writer.writerow(['brand_name','product_name', 'category', 
                         'original_price', 'discount_price',
                         'website'])
        
        
        # Collect information on the product page
        website = 'Saks'
        cat = category
        try:
            brand_name = driver.find_element_by_xpath('//h1[@class="product-brand-name d-none d-sm-block"]').text
        except:
            brand_name = None
            
        try:
            product_name = driver.find_element_by_xpath('//div/h1[@class="product-name h2 d-none d-sm-block"]').text
        except:
            product_name = None
            
        try:
            original_price = driver.find_element_by_xpath('//span[@class="formatted_price bfx-price bfx-list-price"]').text
            original_price = float(re.search('(\d+\.*\d+)',original_price).group(1))
        except:
            original_price = None
            
        try:
            discount_price = driver.find_element_by_xpath('//span[@class="formatted_sale_price formatted_price js-final-sale-price bfx-price bfx-sale-price"]').text
            discount_price = float(re.search('(\d+\.*\d+)',discount_price).group(1))
        except:
            discount_price = None
            
        
        #write in csv file 
        item['website'] = website
        item['brand_name'] = brand_name
        item['product_name'] = product_name
        item['category'] = cat
        item['original_price'] = original_price
        item['discount_price'] = discount_price

        writer.writerow(item.values())

In [48]:
# on each category page, get all the item links
# driver = webdriver.Chrome(r'C:\Users\Chun\chromedriver_win32\chromedriver.exe')
driver.get('https://www.saksfifthavenue.com/product/poupette-st-barth-bety-printed-tiered-dress-0400012349488.html?dwvar_0400012349488_color=AQUA%20ASPEN')
# time.sleep(PAUSE_TIME)
discount_price = driver.find_element_by_xpath('//span[@class="formatted_sale_price formatted_price js-final-sale-price bfx-price bfx-sale-price"]').text